In [ ]:
import time

from ngsolve import *
from ngsolve.webgui import Draw
from netgen.csg import unit_cube
from ngsolve.krylovspace import CGSolver
n = specialcf.normal(3)
def norm(u, Mesh):
    with TaskManager():
        return sqrt(Integrate( InnerProduct(u,u) , Mesh))
def Curl(u):
    if u.dim == 3:
        return CF( (u[2].Diff(y)- u[1].Diff(z), u[0].Diff(z)- u[2].Diff(x), u[1].Diff(x)- u[0].Diff(y)) )
    if u.dim == 9:
        return CF( (Curl(u[0,:]),Curl(u[1,:]),Curl(u[2,:])),dims=(3,3) )
def Inc(u):
    return Curl((Curl(u)).trans)
P_n = OuterProduct(n,n) 
Q_n = Id(3) - OuterProduct(n,n) 
C_n = CF( (0,n[2],-n[1],-n[2],0,n[0],n[1],-n[0],0), dims=(3,3) )
def C(V): 
    return CF( (0,V[2],-V[1],-V[2],0,V[0],V[1],-V[0],0), dims=(3,3) ) 


# Hellinger Reissner Mixed methods for elasticity:

1. Primal
2. Dual 

In [ ]:
def Primal(Maxh,Order):
    mesh = Mesh(unit_cube.GenerateMesh(maxh=Maxh))
    fesHCurl = HCurl(mesh, order=Order, dirichlet= ".*")
    with TaskManager():
        ##########################################################
        #           Coefficient and grid functions               #
        ##########################################################
           
        # continuous functions
        peak = exp(-25*((x-0.5)**2+(y-0.5)**2+(z-0.5)**2))

        # our initail function
        PEAK = CF ( (peak, 0, 0 ), dims=(3,1))
        
        # true curl(gamma)
        #CurlPeak2 = CF ( (0,(25-50*z)*peak, 0 ), dims=(3,1))
        CurlPeak = CF( Curl(PEAK), dims=(3,1) )

        # grid functions 
        gfG = GridFunction(fesHCurl) 
        gfG.Set ( PEAK, bonus_intorder=15, dual=True)

        gfCurlG = GridFunction(fesHCurl)
        gfCurlG.Set(CurlPeak, definedon=mesh.Boundaries(".*"),  bonus_intorder=15, dual=True)
        #print("CurlPeak only on boundaries")
        #Draw (gfCurlG , mesh, clipping=(0,1,1),deformation=False) 

        #gfCurlG2 = GridFunction(fesHCurl)
        #gfCurlG2.Set(CurlPeak2, BND,  bonus_intorder=15, dual=True)
        #Draw (gfCurlG2 , mesh, clipping=(0,1,1),deformation=False) 

        #print("**********************************************************************")
        ##########################################################
        #              Linear and Bilinear forms                 #
        ##########################################################

        u,v = fesHCurl.TnT()
        #some geometrical objects we need

        # Mass matrix
        a = BilinearForm(fesHCurl, symmetric=True)
        a += InnerProduct(u,v)*dx 

        # linear form induced by the metric gfG
        f = LinearForm(fesHCurl)

        # thetrahedron inc part
        f += InnerProduct(curl(gfG),v)*dx        
        # faces part:
        u_n = InnerProduct(gfG,n)
        f += InnerProduct(v,Cross(n,gfG))*dx(element_vb=BND)
        
        ##########################################################
        #           Assemble, compute and print!                 #
        ##########################################################
        pre = Preconditioner(a, "local")
        a.Assemble()
        f.Assemble()

        r = f.vec.CreateVector()
        r.data = f.vec - a.mat * gfCurlG.vec
        

        inverse = CGSolver(a.mat, pre.mat , printrates='\r', maxiter=1000,tol=1e-12)
        gfCurlG.vec.data += inverse * r
        #gfCurlCurlG.vec.data += a.mat.Inverse(freedofs=fesHCurlCurl.FreeDofs(),inverse="sparsecholesky") * r
        

        #print("error of my curl")
        Error = norm(gfCurlG-CurlPeak,mesh)
        #print("Error curl(PEAK)-gfCurlG when order is "+str(Order)+"-->"+str(Error) )

        #print("**********************************************************************")
        #print("CurlPEAK-gfCurlG")
        #Draw (gfCurlG-Curl(PEAK),mesh, draw_surf=True, clipping=(0,1,1), deformation=False)
        #print("CurlPeak")
        #Draw (CurlPeak, mesh, clipping=(0,1,1),name = "Real inc gfG",deformation=False) 
        #print("gfCurlG")
        #Draw (gfCurlG,mesh, draw_surf=True, clipping=(0,1,1), deformation=False)
        #print("**********************************************************************")
        
        #print("CurlPEAK-PEAK2")
        #Draw (CurlPeak2-CurlPeak,mesh, draw_surf=True, clipping=(0,1,1), deformation=False)
        #Draw (CurlPeak2,mesh, draw_surf=True, clipping=(0,1,1), deformation=False)
        #Draw (CurlPeak,mesh, draw_surf=True, clipping=(0,1,1), deformation=False)

        return Error